
# 문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 (LangGraph)
**요구:** MessageState를 사용해 대화형으로 **메뉴/가격/추천** 문의에 응답.  
**핵심:** 의미론적 검색 + 상태 기반 분기 + 구조화 정보 추출.


## 0) 환경 점검

In [1]:

import os, sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0
OPENAI_API_KEY set: True


## 1) 벡터 DB 로드

In [2]:

from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

DB_DIR = Path.cwd() / "db" / "cafe_db"
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.load_local(str(DB_DIR), embeddings=emb, allow_dangerous_deserialization=True)
print("FAISS loaded:", DB_DIR.exists())


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS loaded: True


## 2) 구조화 정보 추출

In [3]:

import re
from langchain.schema import Document

def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    price_match = re.search(r'₩([\d,]+)', content)
    desc_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    return {
        "name": menu_name,
        "price": f"₩{price_match.group(1)}" if price_match else "가격 정보 없음",
        "description": desc_match.group(1).strip() if desc_match else "설명 없음"
    }


## 3) 문의 유형 분류 로직

In [4]:

def classify_intent(text: str) -> str:
    t = (text or "").lower()
    if any(k in t for k in ["추천", "추천해", "인기", "뭐가 좋아", "고르면"]):
        return "recommend"
    if any(k in t for k in ["가격", "얼마", "비싸", "저렴"]):
        return "price"
    return "menu"


## 4) LangGraph — 상태/노드/그래프

In [6]:
from typing import List, Dict, Any
from langgraph.graph import StateGraph, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

def node_classify(state: dict) -> dict:
    user_msg = state["messages"][-1].content
    state["intent"] = classify_intent(user_msg)
    return state

def node_search(state: dict) -> dict:
    user_msg = state["messages"][-1].content
    intent = state.get("intent", "menu")
    if intent == "price":
        docs = vs.similarity_search("메뉴 가격", k=5)
    elif intent == "recommend":
        docs = vs.similarity_search(user_msg, k=3) or vs.similarity_search("인기 메뉴", k=3)
    else:
        docs = vs.similarity_search(user_msg, k=4)
    state["docs"] = docs
    return state

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def node_respond(state: dict) -> dict:
    intent = state.get("intent", "menu")
    user_msg = state["messages"][-1].content
    infos = [extract_menu_info(d) for d in state.get("docs", [])]
    context = "\n".join([f"- {i['name']} | {i['price']} | {i['description']}" for i in infos])

    if intent == "price":
        prompt = f"""가격 문의에 간단하게 답하세요.
질문: {user_msg}
문맥:
{context}
한국어로 2~3문장."""
    elif intent == "recommend":
        prompt = f"""카페 추천을 제안하세요.
질문: {user_msg}
문맥:
{context}
최대 3개 메뉴를 이유와 함께."""
    else:
        prompt = f"""메뉴 정보를 설명하세요.
질문: {user_msg}
문맥:
{context}
가격과 특징 포함, 3문장 이내."""

    ans = llm.invoke(prompt).content
    state["messages"].append(AIMessage(content=ans))
    return state

# ✅ MessagesState 사용
graph = StateGraph(MessagesState)
graph.add_node("classify", node_classify)
graph.add_node("search", node_search)
graph.add_node("respond", node_respond)

graph.set_entry_point("classify")
graph.add_edge("classify", "search")
graph.add_edge("search", "respond")
graph.add_edge("respond", END)

app = graph.compile()


## 5) 테스트

In [7]:

from langchain_core.messages import HumanMessage

tests = [
    "아메리카노 가격 알려줘",
    "달달한 메뉴 추천해줘",
    "콜드브루의 특징은 뭐야?"
]

for t in tests:
    state = {"messages": [HumanMessage(content=t)]}
    final = app.invoke(state)
    print(f"\n== 사용자: {t}")
    print(final["messages"][-1].content)



== 사용자: 아메리카노 가격 알려줘
아메리카노는 일반적으로 3,500원에서 5,000원 사이의 가격으로 제공됩니다. 진한 에스프레소에 뜨거운 물을 추가하여 부드러운 맛을 즐길 수 있는 커피입니다. 카페인 함량이 높아 피로 회복에 도움을 주며, 다양한 디저트와 잘 어울립니다.

== 사용자: 달달한 메뉴 추천해줘
추천하는 달달한 메뉴는 '초코 브라우니'입니다. 가격은 약 5,000원으로, 진한 초콜릿 맛과 부드러운 식감이 특징이며, 바닐라 아이스크림과 함께 제공되어 더욱 풍부한 맛을 즐길 수 있습니다. 또한, 따뜻하게 제공되어 입안에서 녹는 느낌이 일품입니다.

== 사용자: 콜드브루의 특징은 뭐야?
콜드브루는 찬물로 천천히 추출한 커피로, 일반적으로 12시간 이상 우려내어 부드럽고 풍부한 맛을 자랑합니다. 카페인 함량이 높고 쓴맛이 적어, 아이스 음료로 인기가 많습니다. 가격은 보통 4,000원에서 6,000원 사이로, 카페마다 다를 수 있습니다.
